<a href="https://colab.research.google.com/github/Batmaev/qua/blob/main/Permutations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import cirq
except ImportError:
    !pip install --quiet cirq --pre
    import cirq
import sympy
from scipy.optimize import *
import numpy as np

# Задача коммивояжёра

Ответом на задачу является последовательность, в которой мы посещаем города.

Первый город можно выбрать $n$ способами. Если граф полный, то второй город можно выбрать $n-1$ способом, третий $n-2$ способами и так далее, суммарно $n!$ возможных последовательностей городов.

Будем искать незамкнутые пути.

In [ ]:
# n_towns = 4

In [ ]:
def create_cost_matrix(n_towns):
   """Рандомная симметричная матрица с положительными элементами, 
   которая хранит длины рёбер между каждой парой городов"""

   A = np.random.random((n_towns, n_towns))
   A -= np.tril(A)
   
   return A.T + A

In [ ]:
# cost_matrix_4 = create_cost_matrix(4)

cost_matrix_4 = np.array([[0.        , 0.04711181, 0.01197604, 0.85992487],
                          [0.04711181, 0.        , 0.68829915, 0.80606249],
                          [0.01197604, 0.68829915, 0.        , 0.46924138],
                          [0.85992487, 0.80606249, 0.46924138, 0.        ]])

In [ ]:
def cost_of_permutation(cost_matrix, town_sequence):
  cost = 0.0

  for i in range(len(town_sequence)-1):
    cost += cost_matrix [town_sequence[i]] [town_sequence[i+1]]
  
  return cost

#### Аналитическое решение

Переберём все перестановки в лексикографическом порядке.

Будем итерироваться по номеру перестановки. Сначала переведём его в факториальную систему счисления [(factoradic)](https://en.wikipedia.org/wiki/Factorial_number_system). Полученные цифры будут совпадать с [кодом Лемера](https://en.wikipedia.org/wiki/Lehmer_code) перестановки. 

Затем по коду Лемера вычислим саму перестановку и затем её стоимость.

In [ ]:
# Для перевода из десятичной системы счисления в факториальную 
# понадобятся факториалы, поэтому вычислим их заранее:
#
# factorials[n] == n!

factorials = [1]
for i in range(30):
  # Предполагаю, что n_towns < 30
  factorials.append(factorials[i] * (i+1))

In [ ]:
def factoradic_from_perm_number(perm_number : int, factoradic_array):
  """Преобразует число в факториальную систему счисления.
  Записывает полученные цифры в factoradic_array.
  Нулевая цифра в массиве - самая значимая. Последняя цифра всегда равна 0.
  Алгоритм из https://en.wikipedia.org/wiki/Factorial_number_system#Definition """

  N = perm_number
  n_towns = len(factoradic_array)

  for i in range(len(factoradic_array)):
    factoradic_array[i] = N // factorials[n_towns - i - 1]
    N %= factorials[n_towns - i - 1]
  
  return factoradic_array

In [ ]:
def permutation_from_factoradic(nums : np.ndarray):
  """Перезаписывает массив nums.
  Если изначально там находилось число в факториальной системе, то потом там будет находиться перестановка.
  Города в полученной перестановке нумеруются с 0. 
  Алгоритм из https://en.wikipedia.org/wiki/Lehmer_code#Encoding_and_decoding """
  for i in range(len(nums)-2, -1, -1):
    for j in range(i+1, len(nums)):
      if(nums[j] >= nums[i]):
        nums[j] += 1
  return nums

In [ ]:
# Тест из Википедии
permutation_from_factoradic([1, 4, 0, 3, 1, 1, 0]) \
                         == [1, 5, 0, 6, 3, 4, 2]

True

In [ ]:
def analytical(cost_matrix):

  n_towns = len(cost_matrix)

  factoradic_array = np.ndarray(n_towns, int)

  min_cost = np.inf
  opt_perm_number = np.nan

  for perm_number in range(factorials[n_towns]):
    cost = cost_of_permutation(
        cost_matrix,
        permutation_from_factoradic(
            factoradic_from_perm_number(perm_number, factoradic_array)
            )
        )
    if cost < min_cost:
      min_cost = cost
      opt_perm_number = perm_number

  return {"min_cost": min_cost, 
          "order_of_towns": permutation_from_factoradic(
              factoradic_from_perm_number(opt_perm_number, factoradic_array)
            )}

In [ ]:
analytical(cost_matrix_4)

{'min_cost': 0.52832923, 'order_of_towns': array([1, 0, 2, 3])}

## Цепь

In [ ]:
class FactoradicCirquit:
   """
Схема из нескольких пар слоёв Rx и CNOT.
Может подобрать оптимальные углы для гейтов Rx.

3-я цифра ├ 0: ───Rx(a_0)───@───────────────────Rx(a_6)────@───────────────────M──
                            │                              │
    2-ая  ┌ 1: ───Rx(a_1)───X───@───────────────Rx(a_7)────X───@───────────────M──
    цифра |                     │                              │
          └ 2: ───Rx(a_2)───────X───@───────────Rx(a_8)────────X───@───────────M──
                                    │                              │
          ┌ 3: ───Rx(a_3)───────────X───@───────Rx(a_9)────────────X───@───────M──
    1-ая  |                             │                              │
    цифра | 4: ───Rx(a_4)───────────────X───@───Rx(a_10)───────────────X───@───M──
          |                                 │                              │
          └ 5: ───Rx(a_5)───────────────────X───Rx(a_11)───────────────────X───M──

Блоки из кубитов кодируют цифры в факториальной системе счисления.
Закодированная цифра равна сумме единиц, получившихся в результате измерения.
   """

   def __init__(self, n_towns, n_layer_pairs):
      # n_qubits = 1 + 2 + ... + n-1 = (n-1)n/2
      n_qubits = (n_towns - 1) * n_towns // 2

      self.qubits = cirq.LineQubit.range(n_qubits)
      self.circuit = cirq.Circuit()
      self.simulator = cirq.Simulator()

      self._town_sequence = np.ndarray(n_towns, int)

      self._angle_names = [f'a_{i}' for i in range(n_layer_pairs * n_qubits)]
      angle_symbols = sympy.symbols(self._angle_names)


      for layer_pair in range(n_layer_pairs):
          rx_gates = [cirq.rx(angle)(qubit)
                      for (angle, qubit)
                      in zip(angle_symbols[layer_pair*n_qubits : (layer_pair+1)*n_qubits],    self.qubits) ]
          self.circuit.append(cirq.Moment(rx_gates))

          cnot_gates = [cirq.CNOT(q1, q2)
                        for (q1, q2)
                        in zip(self.qubits, self.qubits[1:]) ]
          self.circuit.append(cnot_gates)

      measurements = cirq.measure_each(*self.qubits)
      self.circuit.append(cirq.Moment(measurements))


   def measure(self, angles, repetitions = 1):
      """
      Аргумент 'angles' должен быть массивом длины n_qubits * n_layer_pairs,
      где n_qubits = n_towns * (n_towns - 1) / 2.
      """

      params = cirq.ParamResolver({
         name: value for (name, value) in
         zip(self._angle_names, angles)
      })

      return self.simulator.run_sweep(self.circuit, params, repetitions)[0]


   def town_sequence_from_bits(self, bits):
      """Вычисляет перестановку на основе результатов измерений
      записывает её в self._town_sequence и возвращает её"""

      # Сначала запишем в self._town_sequence номер перестановки 
      # в факториальной системе счисления, а потом вычислим саму перестановку
      # при помощи внешней функции permutation_from_factoradic

      bit_pos = 0
      for digit_number_from_end in range(len(self._town_sequence)):

        # Нулевая цифра с конца равна нулю.
        # Первая цифра с конца меньше либо равна 1, вторая - меньше либо равна 2 и так далее.
        #
        # Для каждой цифры суммируем digit_number_from_end элементов массива bits.

        digit_value = 0
        for _ in range(digit_number_from_end):
          digit_value += bits[bit_pos]
          bit_pos += 1 

        self._town_sequence[- 1 - digit_number_from_end] = digit_value

      return permutation_from_factoradic(self._town_sequence)


   def cost_of_bits(self, bits, cost_matrix):
      "Вычисляет по последовательности битов перестановку и её стоимость"
      return cost_of_permutation(
          cost_matrix,
          self.town_sequence_from_bits(bits)
      )


   def costs(self, angles, cost_matrix, repetitions):
      """Чтобы погрешность среднего была меньше 10^-3, 
      repetitions должно быть порядка миллиона"""

      measurementDataFrame = self.measure(angles, repetitions).data

      return measurementDataFrame.apply(
          self.cost_of_bits, 
          axis = 1, raw = True, args = (cost_matrix,)
          )


   def optimize_without_constraints(self, cost_matrix, 
                                    optimizer = differential_evolution,
                                    repetitions = 100,
                                    **optimizer_kwargs):
      """
      Вызывает функцию, заданную в аргументе optimizer.
      С её помощью подбирает параметры гейтов так,
      чтобы решить задачу коммивояжёра.
      Маршрут вычисляется на основе результатов измерений снова и снова,
      а затем суммарная длина пути усредняется. 
      Именно эту усреднённую длину пути мы пытаемся минимизировать.

      Args:

       cost_matrix - матрица размером n_towns × n_towns такая, что
                     cost_matrix[i][j] есть длина пути между городами i и j

       optimizer - одна из функций https://docs.scipy.org/doc/scipy/reference/optimize.html#global-optimization

       repetitions - число повторений для усреднения коста. 
                     Для градиентных методов нужна высокая точность, 
                     так как они считают разность между близкими величинами.
                     Могут потребоваться миллионы повторений.
      """

      cost_lambda = lambda angles : self.costs(angles, cost_matrix, repetitions).mean()

      if optimizer == basinhopping or optimizer == minimize:
         if 'x0' not in optimizer_kwargs:
            optimizer_kwargs['x0'] = [2 * np.pi * (np.random.random() - 0.5)
                                    for _ in self._angle_names]
      else:
         if 'bounds' not in optimizer_kwargs:
            optimizer_kwargs['bounds'] = [(-np.pi, np.pi) for _ in self._angle_names]

      return optimizer(cost_lambda, **optimizer_kwargs)


   def __str__(self):
      return self.circuit.__str__()

   def _repr_pretty_(self, *args):
      "Text output in Jupyter"
      return self.circuit._repr_pretty_(*args)

In [ ]:
circuitInstance_4_1 = FactoradicCirquit(n_towns = 4, n_layer_pairs = 1)
circuitInstance_4_1

0: ───Rx(a_0)───@───────────────────M───
                │
1: ───Rx(a_1)───X───@───────────────M───
                    │
2: ───Rx(a_2)───────X───@───────────M───
                        │
3: ───Rx(a_3)───────────X───@───────M───
                            │
4: ───Rx(a_4)───────────────X───@───M───
                                │
5: ───Rx(a_5)───────────────────X───M───

#### Существующая погрешность

Оценим стандартное отклонение величины `cost`:

In [ ]:
angles = np.ones(6)
circuitInstance_4_1.costs(angles, cost_matrix_4, repetitions=100).std()

0.3825670685598723

Стандартное отклонение величины `cost_averaged` в $\sqrt{N_\text{repetitions}}$ раз меньше.

Например, при миллионе повторений погрешность величины `cost_averaged` будет около `0.004`. 

С таким количеством повторений дифференциальный спуск работает долго.

#### Допустимый уровень погрешности

Нужно, чтобы погрешность коста была много меньше, чем разность костов между любыми двумя путями:      

$$\sigma_\text{cost_averaged} \ll \underset{i,j}{\min} |\operatorname{cost}_i - \operatorname{cost}_j|$$

Оценим минимальную разность костов между путями.

Кост отдельного ребра лежит от $0$ до $1$. 

Кост пути лежит от $0$ до $n-1$. $\;\;(n = n_\text{towns})$

В этом диапазоне расположено $n!$ возможных путей, соответственно расстояние между ними порядка 

$$\frac{n-1}{n!} \approx \frac{1}{(n-1)!}$$

В итоге

$$\frac{\sigma_\text{cost}}{\sqrt{N_\text{repetitions}}} \ll \frac{1}{(n-1)!} $$

$$N_\text{repetitions} = O\bigl((n-1)!^2\bigr)$$

Ужасно(((

Запустить полный перебор будет быстрее, чем один раз вычислить `cost_averaged`.

#### Испытания

Классические оптимизаторы `scipy.optimize.differential_evolution` и `dual_annealing` всё равно каким-то чудом работают (со 100 повторениями).

Любопытно, что `differential_evolution` (вопреки названию) не использует дифференциирование. 

In [ ]:
# differential_evolution
# 30 sec, success
circuitInstance_4_1.optimize_without_constraints(cost_matrix_4)

     fun: 0.5283292299999991
 message: 'Optimization terminated successfully.'
    nfev: 2439
     nit: 24
 success: True
       x: array([0.04548137, 0.04558345, 0.06161271, 0.11698397, 0.41172122,
       2.95171758])

In [ ]:
# 2 min, success
circuitInstance_4_1.optimize_without_constraints(cost_matrix_4, dual_annealing)

     fun: 0.5283292299999991
 message: ['Maximum number of iteration reached']
    nfev: 14066
    nhev: 0
     nit: 1000
    njev: 295
  status: 0
 success: True
       x: array([ 0.30663331,  0.07381871,  0.13764033, -0.28406106,  2.58826168,
       -3.06467442])

Остальные классические оптимизаторы плохо работают:

In [ ]:
# ~approximately success, 7 min 
circuitInstance_4_1.optimize_without_constraints(cost_matrix_4, minimize, repetitions = 10**4, method = 'nelder-mead')

 final_simplex: (array([[-3.44471701e-02, -3.37047192e-03, -8.47815190e-02,
        -3.68147381e-02,  3.64371586e+00,  3.04323728e+00],
       [-3.44471701e-02, -3.37047192e-03, -8.47815190e-02,
        -3.68147381e-02,  3.64371586e+00,  3.04323728e+00],
       [-3.44471701e-02, -3.37047192e-03, -8.47815190e-02,
        -3.68147381e-02,  3.64371586e+00,  3.04323728e+00],
       [-3.44471701e-02, -3.37047192e-03, -8.47815190e-02,
        -3.68147381e-02,  3.64371586e+00,  3.04323728e+00],
       [-3.44471701e-02, -3.37047192e-03, -8.47815190e-02,
        -3.68147381e-02,  3.64371586e+00,  3.04323728e+00],
       [-3.44471701e-02, -3.37047192e-03, -8.47815190e-02,
        -3.68147381e-02,  3.64371586e+00,  3.04323728e+00],
       [-3.44471701e-02, -3.37047192e-03, -8.47815190e-02,
        -3.68147381e-02,  3.64371586e+00,  3.04323728e+00]]), array([0.53005052, 0.53038633, 0.53108431, 0.53162714, 0.53165076,
       0.53172206, 0.5325661 ]))
           fun: 0.5300505169370482
       messag

In [ ]:
# Fail, 8 min
circuitInstance_4_1.optimize_without_constraints(cost_matrix_4, minimize, 10**5, method='BFGS')

      fun: 1.387791301352183
 hess_inv: array([[ 0.66449409, -0.24582682, -0.13325578, -0.15410054, -0.27027345,
        -0.21901418],
       [-0.24582682,  0.82425257, -0.09273823, -0.11238576, -0.20048564,
        -0.16656984],
       [-0.13325578, -0.09273823,  0.95256417, -0.06061766, -0.11009826,
        -0.09382111],
       [-0.15410054, -0.11238576, -0.06061766,  0.92928332, -0.1244333 ,
        -0.1013265 ],
       [-0.27027345, -0.20048564, -0.11009826, -0.1244333 ,  0.78365465,
        -0.17300688],
       [-0.21901418, -0.16656984, -0.09382111, -0.1013265 , -0.17300688,
         0.86553483]])
      jac: array([98275.10268055, 48354.98488411, 12524.96059544, 42300.74258596,
       92451.10514301, 97143.73923972])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 131
      nit: 1
     njev: 17
   status: 2
  success: False
        x: array([ 1.30533178,  2.62006331,  0.10503384, -0.0166915 ,  2.2282903 ,
       -1.89141241])

In [ ]:
# 12 min, fail
optimizeResult_4_1 = circuitInstance_4_1.optimize_without_constraints(cost_matrix_4, basinhopping, repetitions=1000)
optimizeResult_4_1

                        fun: 1.079744480930002
 lowest_optimization_result:       fun: 1.079744480930002
 hess_inv: array([[1.02472507, 1.20272017, 0.88122054, 0.75914089, 0.04465742,
        0.32792516],
       [1.20272017, 1.68105849, 0.84230359, 1.19271383, 0.35343789,
        0.61936331],
       [0.88122054, 0.84230359, 1.8666406 , 1.08604586, 0.26975885,
        0.54482786],
       [0.75914089, 1.19271383, 1.08604586, 2.22425495, 0.24708063,
        0.59320288],
       [0.04465742, 0.35343789, 0.26975885, 0.24708063, 1.02344534,
        0.11000714],
       [0.32792516, 0.61936331, 0.54482786, 0.59320288, 0.11000714,
        1.28386033]])
      jac: array([2238006.24033037,  235710.78819348,  817634.01667181,
       1581867.47014926,  625223.53183085,  878972.97798556])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 172
      nit: 1
     njev: 23
   status: 2
  success: False
        x: array([-1.7467565 , -0.61127046,  0.35957316,  5.29431684